In [ ]:
# Install Dependencies
!pip install -q timm opencv-python scikit-image albumentations

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Imports
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

import albumentations as A
from albumentations.pytorch import ToTensorV2

from timm import create_model

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# Dataset Constants
CLASSES = [
    'im_Dyskeratotic',
    'im_Koilocytotic',
    'im_Metaplastic',
    'im_Parabasal',
    'im_Superficial_Intermediate'
]

NUM_CLASSES = len(CLASSES)
IMG_SIZE = 224

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_tfms = A.Compose([
    A.Resize(224, 224),

    # Geometry (important for morphology)
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),

    # Cytology-safe color robustness
    A.RandomBrightnessContrast(
        brightness_limit=0.15,
        contrast_limit=0.15,
        p=0.7
    ),
    A.HueSaturationValue(
        hue_shift_limit=5,
        sat_shift_limit=10,
        val_shift_limit=5,
        p=0.5
    ),

    # Prevent over-sharpening
    A.GaussianBlur(blur_limit=3, p=0.15),

    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2()
])

val_tfms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2()
])


In [ ]:
# Custom SIPaKMeD Dataset
class SIPaKMeDDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform

        for idx, cls in enumerate(CLASSES):
            cls_path = os.path.join(root_dir, cls)
            for file in os.listdir(cls_path):
                if file.lower().endswith(".bmp"):
                    self.samples.append((os.path.join(cls_path, file), idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]

        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Stain normalization
        # img = reinhard_normalization(img, TARGET_MEANS, TARGET_STDS)

        if self.transform:
            img = self.transform(image=img)["image"]

        return img, label

In [ ]:
# load Dataset
train_dataset = SIPaKMeDDataset("/content/drive/MyDrive/wiseyak/cervical_cancer/SIPAKMED/Training", train_tfms)
val_dataset   = SIPaKMeDDataset("/content/drive/MyDrive/wiseyak/cervical_cancer/SIPAKMED/Test", val_tfms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))

In [ ]:
# Class-Weighted Loss
labels = [label for _, label in train_dataset.samples]

weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

class_weights = torch.tensor(weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)

In [ ]:
# Load MobileViT-S
model = create_model(
    "mobilevit_s",
    pretrained=True,
    num_classes=NUM_CLASSES
).to(device)

print(model)

In [ ]:
for name, param in model.named_parameters():
    print(name)


In [ ]:
# Freeze entire model first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze classification head (timm-safe)
if hasattr(model, "head"):
    for param in model.head.parameters():
        param.requires_grad = True
else:
    raise RuntimeError("Model head not found - check timm version")


In [ ]:
# Optimizer (Warm-Up Phase)
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=3e-4,
    weight_decay=1e-4
)

In [ ]:
# Training & Validation Functions
def train_epoch(model, loader):
    model.train()
    total_loss = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


@torch.no_grad()
def validate(model, loader):
    model.eval()
    correct, total = 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        pred = out.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    return correct / total

In [ ]:
# Phase 1: Classifier Warm-Up
for epoch in range(5):
    loss = train_epoch(model, train_loader)
    acc = validate(model, val_loader)
    print(f"[Warm-Up] Epoch {epoch+1} | Loss: {loss:.4f} | Acc: {acc:.4f}")

In [ ]:
# Unfreeze Entire Model
for param in model.parameters():
    param.requires_grad = True

In [ ]:
# Optimizer + Scheduler (Fine-Tuning)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

In [ ]:
# Phase 2: Full Fine-Tuning
for epoch in range(20):
    loss = train_epoch(model, train_loader)
    acc = validate(model, val_loader)
    scheduler.step()

    print(f"[Fine-Tune] Epoch {epoch+1} | Loss: {loss:.4f} | Acc: {acc:.4f}")

In [ ]:
# Save Model (Inference-Safe)
torch.save({
    "model_state": model.state_dict(),
    "classes": CLASSES
}, "mobilevit_s_sipakmed_stain_normalized.pth")

In [ ]:
# Infer
import os
import random
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import timm

# ----------------------------
# CONFIG
# ----------------------------
TEST_DIR = "/content/drive/MyDrive/wiseyak/cervical_cancer/SIPAKMED/Test"
MODEL_PATH = "/content/mobilevit_s_sipakmed_stain_normalized.pth"
BATCH_SIZE = 32
NUM_WORKERS = 2

CLASSES = [
    'im_Dyskeratotic',
    'im_Koilocytotic',
    'im_Metaplastic',
    'im_Parabasal',
    'im_Superficial_Intermediate'
]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

# ----------------------------
# TRANSFORMS (match your val_tfms)
# ----------------------------
val_tfms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

# ----------------------------
# DATASET + DATALOADER
# ----------------------------
test_dataset = datasets.ImageFolder(TEST_DIR, transform=val_tfms)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=NUM_WORKERS)

print("Test samples:", len(test_dataset))
assert test_dataset.classes == CLASSES, "Dataset class order mismatch!"

# ----------------------------
# LOAD MODEL
# ----------------------------
print("Loading checkpoint...")
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)

# If checkpoint wrapped
if "model_state" in checkpoint:
    state_dict = checkpoint["model_state"]
    print("Loaded model_state from checkpoint")
else:
    state_dict = checkpoint
    print("Loaded raw state_dict")

model = timm.create_model("mobilevit_s", pretrained=False, num_classes=len(CLASSES))
model.load_state_dict(state_dict, strict=False)

model = model.to(DEVICE)
model.eval()

print("Model loaded successfully!")


# ----------------------------
# INFERENCE
# ----------------------------
all_preds = []
all_labels = []
correct = 0
total = 0

annot_images = []   # store images for later visualization
annot_preds = []
annot_true = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (preds == labels).sum().item()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # store sample images for annotation
        for i in range(imgs.size(0)):
            if len(annot_images) < 2000:  
                annot_images.append(imgs[i].cpu())
                annot_preds.append(preds[i].cpu().item())
                annot_true.append(labels[i].cpu().item())

# ----------------------------
# METRICS
# ----------------------------
wrong = total - correct
accuracy = 100.0 * correct / total

print("\nRESULTS")
print("---------------------------------")
print(f"Total Images : {total}")
print(f"Correct      : {correct}")
print(f"Wrong        : {wrong}")
print(f"Accuracy     : {accuracy:.2f}%\n")

print("Classification Report")
print("---------------------------------")
print(classification_report(all_labels, all_preds, target_names=CLASSES))

print("Confusion Matrix")
print("---------------------------------")
print(confusion_matrix(all_labels, all_preds))

# ----------------------------
# SHOW 40 RANDOM ANNOTATED IMAGES
# ----------------------------
print("\nShowing 40 random annotated samples...")

indices = list(range(len(annot_images)))
random.shuffle(indices)
indices = indices[:40]

fig = plt.figure(figsize=(18, 18))

for i, idx in enumerate(indices):
    img = annot_images[idx]
    img = img.permute(1, 2, 0).numpy()
    img = np.clip((img * 0.229 + 0.485), 0, 1)  # unnormalize

    true_label = CLASSES[annot_true[idx]]
    pred_label = CLASSES[annot_preds[idx]]

    plt.subplot(8, 5, i + 1)
    plt.imshow(img)
    color = "green" if true_label == pred_label else "red"
    plt.title(f"T:{true_label}\nP:{pred_label}", color=color, fontsize=9)
    plt.axis("off")

plt.tight_layout()
plt.savefig("sipakmed_test_predictions_grid.png", dpi=300)
plt.show()

print("\nSaved annotated grid as sipakmed_test_predictions_grid.png")
